# Clash of Clans CWL
## Mid CWL Tracking


This is the workspace to QA data between pipeline and dataframe. All cwl must be finished before transformation.


* Your Name: Nolan Clark
* Name of API: Clash of Clans API
* Link to main page of API documentation: https://developer.clashofclans.com/#/documentation

In [1]:
# Imports go here
## these are imported in the CWL_Functions file
# import requests
import pandas as pd
import MyKeys
import json
import seaborn as sns
import sklearn
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import CWL_Functions as cf
import requests
from collections import Counter, defaultdict
import time

# API Token used with the documented Authorization header. 
# The headers are the necessary params of each endpoint
MyToken = MyKeys.MyToken
headers = {'authorization': 'Bearer '+(MyToken), 'Accept': 'application/json'}
base = 'https://api.clashofclans.com/v1/'

In [67]:
leagues = [ 'Gold League III',
            'Gold League II',
            'Gold League I',
            'Crystal League III',
            'Crystal League II',
            'Crystal League I',
            'Master League III',
            'Master League II',
            'Master League I',
            'Champion League III',
            'Champion League II',
            'Champion League I']

In [3]:
# open JSON file to update with current CWL
with open("PipelineRunOne.json") as fp:
    LeagueBins = json.load(fp)

In [4]:
{key: (len(LeagueBins[key])) for key in  LeagueBins.keys()}

{'Bronze League III': 0,
 'Bronze League II': 0,
 'Bronze League I': 0,
 'Silver League III': 28,
 'Silver League II': 0,
 'Silver League I': 140,
 'Gold League III': 168,
 'Gold League II': 840,
 'Gold League I': 1764,
 'Crystal League III': 3528,
 'Crystal League II': 5096,
 'Crystal League I': 8596,
 'Master League III': 6692,
 'Master League II': 3556,
 'Master League I': 2688,
 'Champion League III': 2100,
 'Champion League II': 1484,
 'Champion League I': 448}

In [66]:
dud_bin = {
            'Bronze League III': [],
            'Bronze League II': [],
            'Bronze League I': [],
            'Silver League III': [],
            'Silver League II': [],
            'Silver League I': [],
            'Gold League III': [],
            'Gold League II': [],
            'Gold League I': [],
            'Crystal League III': [],
            'Crystal League II': [],
            'Crystal League I': [],
            'Master League III': [],
            'Master League II': [],
            'Master League I': [],
            'Champion League III': [],
            'Champion League II': [],
            'Champion League I': []}

In [69]:
# Test all warTags states as warEnded before transforming into dataframe

start=time.time()

c = {'warEnded':0}
#leagues = ['Crystal League III']
lastEnd = time.localtime()

for league in leagues:
    print(league)
    for warTag in LeagueBins[league]:
        try:
            roundJSON = cf.cwlRound(warTag)
            c[roundJSON['state']]+=1
        except KeyError:
            try:
                flag = time.strptime(roundJSON['endTime'], '%Y%m%dT%H%M%S.%fZ')
                if lastEnd < flag:
                    lastEnd = flag
                    print('Final end updated.')
                pass
            except KeyError:
                dud_bin[league].append(warTag)
                print('dud found:  '+warTag)
                pass
    else:
        continue
    break

    
end=time.time()
print(end-start)

Gold League III
Gold League II
Gold League I
Crystal League III
dud found:  #88UU8QPJY
dud found:  #88UU8QL0R
dud found:  #88UU8QGR9
dud found:  #88UU8QQPV
dud found:  #88V0VQ2QC
dud found:  #88V0VQP8Y
dud found:  #88V0VQ8U2
dud found:  #88V0VQYLR
dud found:  #88V9RC9CC
dud found:  #88V9RCY82
dud found:  #88V9RCLLY
dud found:  #88V9RCQJR
dud found:  #88VLLGPJV
dud found:  #88VLLR8QQ
dud found:  #88VLLR9CC
dud found:  #88VLLGL29
dud found:  #88VR8Y0G2
dud found:  #88VR8Y2UY
dud found:  #88VR8Y98R
dud found:  #88VR8YPLV
dud found:  #88VCV28LY
dud found:  #88VCV29JR
dud found:  #88VCV2Y0V
dud found:  #88VCV2LY9
dud found:  #88VVJY2UY
dud found:  #88VVJY98R
dud found:  #88VVJYPLV
dud found:  #88VVJYQ2Q
Crystal League II
dud found:  #2VVUPR02V
dud found:  #2VVUPGCQY
dud found:  #2VVUPGJ92
dud found:  #2VVUPGUCR
dud found:  #80022YC89
dud found:  #80022YULQ
dud found:  #80022YVJC
dud found:  #80022L222
dud found:  #8009CPUUC
dud found:  #8009CY092
dud found:  #8009CY2QY
dud found:  #8009CY8C

In [105]:
{key: (len(dud_bin[key])) for key in  dud_bin.keys()}

{'Bronze League III': 0,
 'Bronze League II': 0,
 'Bronze League I': 0,
 'Silver League III': 0,
 'Silver League II': 0,
 'Silver League I': 0,
 'Gold League III': 0,
 'Gold League II': 0,
 'Gold League I': 0,
 'Crystal League III': 28,
 'Crystal League II': 56,
 'Crystal League I': 84,
 'Master League III': 56,
 'Master League II': 0,
 'Master League I': 0,
 'Champion League III': 0,
 'Champion League II': 0,
 'Champion League I': 0}

In [104]:
vLeagueBins = {
            'Bronze League III': [],
            'Bronze League II': [],
            'Bronze League I': [],
            'Silver League III': [],
            'Silver League II': [],
            'Silver League I': [],
            'Gold League III': [],
            'Gold League II': [],
            'Gold League I': [],
            'Crystal League III': [],
            'Crystal League II': [],
            'Crystal League I': [],
            'Master League III': [],
            'Master League II': [],
            'Master League I': [],
            'Champion League III': [],
            'Champion League II': [],
            'Champion League I': []}

In [106]:
for league in leagues:
    filtered_list=list(filter(lambda i: i not in dud_bin[league], LeagueBins[league]))
    vLeagueBins[league] = filtered_list

#  Post cwl check json

In [112]:
# write the updated file
with open("PipelineRunOneVERIFIED.json","w") as write_file:
    json.dump(vLeagueBins, write_file)